# Assignment 4b - Graph Convolutional Networks
## Deep Learning Course - Vrije Universiteit Amsterdam, 2024

#### Instructions on how to use this notebook:

This notebook is hosted on Google Colab. To be able to work on it, you have to create your own copy. Go to *File* and select *Save a copy in Drive*.

You can also avoid using Colab entirely, and download the notebook to run it on your own machine. If you choose this, go to *File* and select *Download .ipynb*.

The advantage of using Colab is that you can use a GPU. You can complete this assignment with a CPU, but it will take a bit longer. Furthermore, we encourage you to train using the GPU not only for faster training, but also to get experience with this setting. This includes moving models and tensors to the GPU and back. This experience is very valuable because for many interesting models and large datasets (like large CNNs for ImageNet, or Transformer models trained on Wikipedia), training on GPU is the only feasible way.

The default Colab runtime does not have a GPU. To change this, go to *Runtime - Change runtime type*, and select *GPU* as the hardware accelerator. The GPU that you get changes according to what resources are available at the time, and its memory can go from a 5GB, to around 18GB if you are lucky. If you are curious, you can run the following in a code cell to check:

```sh
!nvidia-smi
```

Note that despite the name, Google Colab does  not support collaborative work without issues. When two or more people edit the notebook concurrently, only one version will be saved. You can choose to do group programming with one person sharing the screen with the others, or make multiple copies of the notebook to work concurrently.

**Submission:** Upload your notebook in .ipynb format to Canvas. The code and answers to the questions in the notebook are sufficient, no separate report is expected. 

In [ ]:
!nvidia-smi

### Introduction

Graphs are very useful data structures that allow us to represent sets of entities and the way they are related among each other. In a graph, entities are also known as *nodes*, and any link between entities is also called an *edge*.

Examples of real world objects that can be modeled as graphs are social networks, where entities are people and relations denote friendship; and molecules, where entities are atoms and relations indicate a bond between them.

There has been increased interest in the recent years in the application of deep learning architectures to graph-structured data, for tasks like predicting missing relations between entities, classifying entities, and classifying graphs. This interest has been spurred by the introduction of Graph Convolutional Networks (GCNs).

In this assignment, you will implement and experiment with one of the first versions of the GCN, proposed by Thomas Kipf and Max Welling in their 2017 paper, [Semi-supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907). In particular, the goals of this assignment are to

- Understand how GCNs are formulated
- Implement the GCN using PyTorch
- Train and evaluate a model for semi-supervised node classification in citation networks
- Train and evaluate a model for binary classification of molecules

### Representing graphs

Suppose we have the following graph:

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/01-graph.png" width="200">

This is an undirected graph (since the edges have no specified direction) with 4 nodes. One way to represent the connectivity structure of the graph is by means of the **adjacency matrix**. The $i$-th row of the matrix contains a 1 in the $j$-th column, if nodes $i$ and $j$ are connected. For an undirected graph like the one above, this means that the adjacency matrix

- Is symmetric (e.g. an edge between 0 and 2 is equivalent as an edge between 2 and 0)
- Is square, of size $n\times n$ where $n$ is the number of nodes

The adjacency matrix for the graph above is then the following:

$$
A =
\begin{bmatrix}
0 & 0 & 1 & 0 \\ 
0 & 0 & 1 & 0 \\
1 & 1 & 0 & 1 \\
0 & 0 & 1 & 0
\end{bmatrix}
$$

A second matrix of interest is the **degree matrix**. This is a diagonal matrix where the $i$-th element of the diagonal indicates the number of edges connected to node $i$. Note that these can be obtained from $A$ by summing across the columns, or the rows. For our example, the degree matrix is

$$
D = \begin{bmatrix}
1 & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 \\
0 & 0 & 3 & 0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

For specific applications, each node in the graph will have an associated vector of features $x\in\mathbb{R}^c$. If our graph is a social network, then the vector of features can contain information like age, location, and musical tastes, in a specific numeric format. In the case of a molecule, the node could represent an atom and have features like the atomic mass, etc. We can lay out the features in a matrix $X\in\mathbb{R}^{n\times c}$, so that the feature vector for node $i$ is in the $i$-th row.

### Loading a citation network

To move to a real world example, we will start with the Cora dataset. This dataset represents a citation network, where nodes are scientific publications, edges denote citations between them, and features are a [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) extracted from their contents.

This graph contains labels for nodes, that represent a specific topic. We will use these for a node classification task.

To easily load it, we will use [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/index.html) (PyG), a deep learning library for graph-structured data.

In [29]:
# Install PyTorch Geometric
import torch
try:
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version.
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    %pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    %pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    %pip install torch-geometric
    import torch_geometric

We can now use the library to download and import the dataset. Initializing the `Planetoid` class returns a `Dataset` object that can contain multiple graphs. In this task we will only use the `Cora` dataset (the citation network) and hence, we will select only the first element.

In [30]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='data/Planetoid', name='Cora')
data = dataset[0]


#### Question 1 (0.25 pt)

The `data` object is an instance of the `Data` class in PyG. Check the [documentation](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html) and report the following properties of the graph:

- Number of nodes
- Number of edges 
- The dimension $c$ of the feature vectors $x\in\mathbb{R}^c$
- The number of targets for the classification task

In [ ]:
nnodes = data.num_nodes
in_features = data.num_node_features
# Also possible
assert (nnodes, in_features) == data.x.size()

nedges = data.num_edges

ntargets = len(set(map(int, data.y)))
# Also possible
assert ntargets == dataset.num_classes

print(f"""
Number of nodes: {nnodes}
Number of features: {in_features}
Number of edges: {nedges}
Number of targets: {ntargets}
""")

#### Question 2 (0.25 pt)

In PyG, edges are provided in a tensor of shape (2, number of edges). You can access it via `data.edge_index`. Each column in this tensor contains the IDs for two nodes that are connected in the graph.

We saw that in an undirected graph, an edge between nodes $i$ and $j$ adds a value of 1 to positions $(i, j)$ and $(j, i)$ of the adjacency matrix. Is this also true for the edge index? That is, if there is an edge $(i, j)$ in `data.edge_index`, is there also an edge for $(j, i)$? This is important to know for the next steps of the implementation.

In [ ]:
indices = data.edge_index.t()
is_true = all([torch.tensor([j, i]) in indices for i, j in indices])
print(f"The answer is: {'YES' if is_true else 'NO'}")

#### Question 3 (0.5 pt)

In graphs, especially large ones, the adjacency matrix is **sparse**: most entries are zero. Sparse matrices allow for efficient storage and computation.

To prepare and pre-process sparse matrices, we will use [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html). Once the matrices are ready, we will convert them to PyTorch tensors.

We will use the [Sparse COO format](https://en.wikipedia.org/wiki/Sparse_matrix#Coordinate_list_(COO)). We encourage you to first get familiar with how it works after continuing with the assignment.

- Use the [`scipy.sparse.coo_matrix()`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) function to build the adjacency matrix. Think of what arguments are needed, and how you can obtain them from the graph data loaded above.
- Use the `sum()` method of sparse matrices, together with `scipy.sparse.diags()`, to compute the degree matrix using the definition above.

Both resulting matrices must be sparse of type `float32`.

In [33]:
import scipy
import scipy.sparse

row = data.edge_index[0, :]
col = data.edge_index[1, :]
assert len(row) == len(col) == nedges

# We just put ones for neighbours
values = torch.ones(nedges)

shape = (nnodes, nnodes)

adj_mat = scipy.sparse.coo_matrix((values, (row, col)), shape=shape)

You might wonder why we suggest to use a scipy sparse matrix, while also PyTorch supports them. The reason is that in the next step, we will be multiplying two sparse matrices, an operation not supported in PyTorch. PyTorch only allows multiplying a sparse matrix with a dense one, something which we will be doing at a later stage.



### The Graph Convolutional Network

The goal of the graph convolution is to take the feature vectors of all nodes $X\in\mathbb{R}^{n\times c}$, and propagate them along the existing edges, to obtain updated representations $Z\in\mathbb{R}^{n\times d}$.


The GCN is initially motivated as performing a convolution, similarly as it is done in CNNs for images, for graph-structured data. In Kipf and Welling (2017), a theoretical derivation leads to the following formula:

$$
Z = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}XW
$$

where

- $W\in\mathbb{R}^{c\times d}$ is a matrix of parameters to be learned via gradient descent
- $\tilde{A} = A + I_n$, where $I_n$ is an $n\times n$ identity matrix
- $\tilde{D}$ is the degree matrix computed with $\tilde{A}$ as the adjacency matrix

If we define $\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$, the graph convolution can be written as $Z = \hat{A}XW$.

#### Question 4 (0.25 pt)

Given the formula for the GCN, explain why it operates by propagating feature vectors across the graph. To answer this, it might be useful to recall the definitions of the adjacency and degree matrices, and how they are involved in the formula.

The multiplication of the **adjacency matrix** (or some function of it) with the matrix $X$ **has the effect of summing up feature vectors for all neighbouring nodes**. This is effectively propagating feature vectors across the graph. The **degree matrix** provides a normalization constant for each feature vector, just to keep the scale of the features.

To give a more precise intuiton, it is helpful to write the formula above in vector form ([source](https://tkipf.github.io/graph-convolutional-networks/)):
$$
    z_{i} = \sum_{j \in \mathcal{N}_i} \frac{1}{c_{ij}}x_{j}W
$$
where we identify $x_j$ as the feature vectors (rows of $X$), $\mathcal{N}_i$ is the set of neighbours of node $i$ and $c_{ij}$ is the normalization constant. The neighbours of the node $i$ are encoded as ones in the $i$-th row of the adjacency matrix $A$. $\tilde{A}$ encodes the same information as $A$ plus adding self-loops. $\tilde{D}$ is the degree matrix (taking into account self-loops), constructed as:
$$
    \tilde{D}_{ii} = \sum_j \tilde{A}_{ij}
$$
which effectively normalizes the summed-up feature vectors according to the number of neighbors of each node. 

More generally we have ,for each layer $l$, the activations (with non-linearity $\sigma$): 
$$
H^{(l+1)} = σ(\tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}H^{(l)}W^{(l)})
$$
where $H^{(0)} = X$. From this it's clear how for each layer $l$, after we transform linearly the node features with $H^{(l)}W^{(l)} = M^{(l)}$, we propagate the transformed node features $M$ by one step over through the edges of the graph by multipling $M$ with the normalized adjacency matrix $\hat{A}$. In fact, $\hat{A}$ has on each row(representing a single node) non-zero values only for the adjacent nodes. Therefore it selects exclusively the values of the latter for each column of $M$ through the matrix multiplication. 

#### Question 5 (0.5 pt)

Compute the **normalized adjacency matrix** $\hat{A}$. The result should be a sparse matrix.

In [34]:
# Add self-loops. This will turn A into \tilde{A}
adj_mat.setdiag(1)

# Sum along columns to get the diagonal
diag = adj_mat.sum(axis=1).A1

# Compute degree matrix
d = scipy.sparse.diags_array(diag, shape=(nnodes, nnodes))

# Compute \hat{A}
adj_mat_norm = ((d ** -(1/2)) @ adj_mat @ (d ** -(1/2))).tocoo()

#### Question 6 (0.5 pt)

So far we have used scipy to build and compute sparse matrices. Since we want to train a GCN with PyTorch, we need to convert $\hat{A}$ into a sparse PyTorch tensor. You can do this with the [`torch.sparse_coo_tensor()`](https://pytorch.org/docs/stable/generated/torch.sparse_coo_tensor.html) function, making sure to specify `torch.float` as the type.

In [35]:
import numpy as np

assert adj_mat_norm.shape == (nnodes, nnodes)

coords = np.array(adj_mat_norm.coords)
values = adj_mat_norm.data

adj_mat_norm_torch = torch.sparse_coo_tensor(indices=coords, values=values, size=(nnodes, nnodes), dtype=torch.float)

#### Question 7 (0.5 pt)

We now have all the ingredients to build a GCN layer. Implement a class (inheriting from `torch.nn.Module`) with a learnable matrix of weights $W\in\mathbb{R}^{c\times d}$. Make sure to

- Call this class `GCNLayer`
- The `__init__()` constructor should take as argument the number of input and output features.
- Use `torch.nn.init.kaiming_uniform_` to initialize $W$.
- Define the `forward` method, which takes as input $X$ and $\hat{A}$ and returns $Z$. Note that multiplications involving the sparse matrix $\hat{A}$ have to be done with `torch.spmm`. 

Once you have implemented the class, instantiate a layer with the correct number of input features for the Cora dataset, and a number of output features of your choice. Do a forward pass and report the shape of the output tensor.

In [ ]:
import torch.nn.functional as F
from torch.nn.parameter import Parameter

class GCNLayer(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weights = Parameter(torch.empty(in_features, out_features))
        self.bias = Parameter(torch.zeros(out_features))
        torch.nn.init.kaiming_uniform_(self.weights.T, nonlinearity="relu")
    
    def forward(self, input_m, adjacency_m):
        assert adjacency_m.size(1) == input_m.size(0)
        return torch.spmm(adjacency_m, F.linear(input_m, self.weights.T)) + self.bias

out_features = 200
gcn_layer = GCNLayer(in_features=in_features, out_features=out_features)

# Forward pass
output = gcn_layer(data.x, adj_mat_norm_torch)
print(f"The shape of the output is: {output.size()}")

#### Question 8 (0.5 pt)

As we have seen so far, the GCN layer implements a special type of linear transformation of the inputs. However, it is often beneficial in deep learning to stack multiple, non-linear transformations of the input features. Implement a second module class for a model with two GCN layers (use the module you implemented in the previous question).

- Call this class `GCN`
- The constructor must now take as input the number of input features, the output dimension of the first layer (this is the hidden layer), and the output dimension of the output layer.
- In the forward pass, add a ReLU activation function after the first layer.

In [37]:
class GCN(torch.nn.Module):
    def __init__(self, in_features, n_hidden, n_output):
        super().__init__()
        self.gcn_layer_1 = GCNLayer(in_features=in_features, out_features=n_hidden)
        self.gcn_layer_2 = GCNLayer(in_features=n_hidden, out_features=n_output)

    def forward(self, x, adjacency):
        x = self.gcn_layer_1(x, adjacency)
        x = F.relu(x)
        x = self.gcn_layer_2(x, adjacency)
        return x

### GCNs for semi-supervised node classification

Now that we have a GCN with two layers, we can test its performance in a node classification task. We will pass the input node features $X$ through the GCN layers, and the output will be of size $n\times k$ where $k$ is the number of classes (which you found in question 1). The label denotes the topic an article in the citation network belongs to (e.g. physics, computer science, etc.).

#### Question 9 (1.5 pt)

Note that the `data` object contains all labels (for all splits) in `data.y`, and binary masks for the train, validation, and test splits in `data.train_mask`, `data.val_mask`, and `data.test_mask`, respectively. These masks are the same size as `data.y`, and indicate which labels belong to which split.

- Create a GCN with two layers (using the class from the previous question), with 32 as the hidden dimension, and the number of output features equal to the number of classes in the Cora dataset.

- Use the Adam optimizer with a learning rate of 0.01.

- Implement a training loop for the GCN. At each step, pass $X$ and $\hat{A}$ to the GCN to obtain the logits. Compute the mean cross-entropy loss **only for the training instances**, using the binary masks.

- After each training step, evaluate the accuracy for the validation instances.

- Train for 100 epochs. Once training is finished, plot the training loss and validation accuracy (in a graph in function of the epoch number), and report the accuracy in the test set.

You should obtain an accuracy over 75% on both the validation and test sets. You can also compare your results with the original paper, which also contains results for the Cora dataset. Give a brief discussion on the results of your experiments.

Note that in contrast with other tasks, like image classification on some datasets, we don't use mini-batches here. The whole matrix of features and the adjacency is passed to the GCN in one step.

In [ ]:
# We are taking inspiration from the PyG tutorial: https://pytorch-geometric.readthedocs.io/en/latest/get_started/introduction.html

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gcn = GCN(in_features=in_features, n_hidden=32, n_output=ntargets).to(device)

optimizer = torch.optim.Adam(gcn.parameters(), lr=0.01)  #, weight_decay=5e-4) (?)
criterion = torch.nn.CrossEntropyLoss()

# Store data
epochs = 100
loss_train = torch.zeros(epochs)
loss_val = torch.zeros_like(loss_train)
acc_train = torch.zeros_like(loss_train)
acc_val = torch.zeros_like(loss_train)

# Move data to gpu
X, t = data.x.to(device), data.y.to(device)
adj = adj_mat_norm_torch.to(device)

size_train = int(data.train_mask.sum())
size_val = int(data.val_mask.sum())

for e in range(epochs):
    gcn.train()
    print(f"Epoch {e+1}\n-------------------------------")
    optimizer.zero_grad()

    ### Training set
    # Forward
    out = gcn(X, adj)[data.train_mask]
    target = t[data.train_mask]

    # Backpropagate
    loss = criterion(out, target)
    loss.backward()
    optimizer.step()

    # Metrics
    loss_train[e] = loss.item() # / size_train
    ncorrect = (out.argmax(1) == target).type(torch.float).sum().item()
    acc_train[e] = ncorrect / size_train

    ### Validation set
    gcn.eval()
    with torch.no_grad():
        # Forward
        out_val = gcn(X, adj)[data.val_mask]
        target_val = t[data.val_mask]

        # Metrics
        loss_val[e] = criterion(out_val, target_val).item()#  / size_val
        ncorrect = (out_val.argmax(1) == target_val).type(torch.float).sum().item()
        acc_val[e] = ncorrect / size_val

    print(f"Training loss: {loss_train[e]:>5f} \n Training accuracy: {acc_train[e]:>5f}\n Validation accuracy: {acc_val[e]:.4f}")

print("Done!")

In [ ]:
### Training set
with torch.no_grad():
    test_out = gcn(X, adj)[data.test_mask]
    test_target = t[data.test_mask]
    test_accuracy = (test_out.argmax(1) == test_target).type(torch.float).mean().item()
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(14, 6))
ax[0].plot(range(epochs), loss_train, label="Training loss")
ax[0].plot(range(epochs), loss_val, label="Validation loss")
ax[0].set_title("Loss per epoch")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].legend()

ax[1].plot(range(epochs), acc_train, label="Acc. train")
ax[1].plot(range(epochs), acc_val, label="Acc. val")
ax[1].set_title("Accuracy per epoch")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Accuracy")
ax[1].legend()

plt.show()

#### Question 10 (0.5 pt)

The paper introduces GCNs as a way to solve a *semi-supervised* classification problem.

- What makes this problem semi-supervised?
- What is the proportion of labeled data used for training with respect to labeled data in the validation and test sets? What is difference in this context with other benchmark tasks in machine learning, like image classification with MNIST?
- Why do you think the GCN performs well in this semi-supervised scenario?

In [ ]:
# Your answer here
unique_labels, counts = torch.unique(data.y, return_counts=True)
print("Unique labels:", unique_labels)
print("Counts per label:", counts)

In [ ]:
# count number of train values
num_train_nodes = torch.sum(data.train_mask).item()
print("Number of training nodes:", num_train_nodes)

num_val_nodes = torch.sum(data.val_mask).item()
print("Number of validation nodes:", num_val_nodes)

num_test_nodes = torch.sum(data.test_mask).item()
print("Number of test nodes:", num_test_nodes)


In [ ]:
print("Number of training nodes per class:")
for label in unique_labels:
    num_train_nodes_per_class = torch.sum((data.y == label) & data.train_mask).item()
    print(f"Class {label}: {num_train_nodes_per_class}")

In [ ]:
num_train_nodes = torch.sum(data.train_mask).item()
num_val_nodes = torch.sum(data.val_mask).item()
num_test_nodes = torch.sum(data.test_mask).item()
total_nodes = num_train_nodes + num_val_nodes + num_test_nodes
total_graph_nodes = data.val_mask.shape[0]


print(f"Proportion of training data: {num_train_nodes / total_nodes:.4f}")
print(f"Proportion of validation data: {num_val_nodes / total_nodes:.4f}")
print(f"Proportion of test data: {num_test_nodes / total_nodes:.4f}")


print(f"Proportion of training data considering all nodes: {num_train_nodes / total_graph_nodes:.4f}")
print(f"Proportion of validation data considering all nodes: {num_val_nodes / total_graph_nodes:.4f}")
print(f"Proportion of test data considering all nodes: {num_test_nodes / total_graph_nodes:.4f}")


**1)**

What makes this problem semi-supervised is the fact that we use only 20 labeled examples per class for the training process. I imagine this to be common for ML tasks involving graphs as usually we have only have one large knowledge-base graph for which we want to extract additional data or predict missing links, that we do not already have with the exception of some examples.

**2)** 

The exact proportion of training, validation and test data is 8.5%, 30.5% and 61% respectively, out of a total size of 1640 nodes considered by any of the masks. Intead, taking into account the total number of nodes in the graph, the proportions are 0.05%, 0.19% and 37% respectively. These proportions are far different with the ones that are commonly seen in other ML tasks in which we have tens of thousands of training data points as for the MNIST dataset. The latter has 60,000 images in the training set and 10,000 in the test set.

**3)** 

We report two main reasons that could explain why GCN perform well in this semi-supervised scenario.

The first one, as outlined [here](https://tkipf.github.io/graph-convolutional-networks/), revolves round the fact that we could see GCNs as a generalized and differentiable version of the Weisfeiler-Lehman algorithm for graphs. The latter, similarly to GCNs, for every step updates each node based on a summation of the features of all the adjacent nodes. After k steps the nodes can be clusterized based on their newly generated features.

The second reason it's centered around the fact that graph data is fundamentally structured. On the other hand, for example, image data can be represented as points in a really high dimentional space for wich complex transformations have to be learned to map these points to the correct classes, because of this a higher amount of data is needed. Graph data, instead , already contains much of the information needed within their morphology, defined by the edges, and little transformations are needed, which can be effectively learned with few examples. 

Threfore, we could say that the main strenght of GCN is that they take advantage of this structure at every step of the computation through the adjacency matrix.




### Loading a dataset of proteins

In the previous sections you learned how to pass the adjacency matrix of a graph with a couple of thousand of nodes, to classify each node with a particular label. A different and useful application of GCNs is graph classification.

In contrast with the previous part, where there was a single, big graph, in graph classification we have multiple graphs, and each graph can be assigned a label. In this part of the assignment you will implement a classifier for proteins.

[Proteins](https://en.wikipedia.org/wiki/Protein_(nutrient)) are parts of the buildings block of life. They consist of chains of amino acids, and can take many shapes. In the PROTEINS dataset, proteins are represented as graphs, where the nodes are amino acids, and an edge between them indicates that they are 6 [Angstroms](https://en.wikipedia.org/wiki/Angstrom) apart. All graphs have a binary label, where 1 means that the protein is not an enzyme.

We will start by loading and examining this dataset.

In [18]:
from torch_geometric.datasets import TUDataset
dataset = TUDataset(root='data/TU', name='PROTEINS', use_node_attr=True)

#### Question 11 (0.25 pt)

Unlike in the previous part, where we selected the first element returned by the loading function, note that here we get all the elements returned by `TUDataset()`. `dataset` is an interable object, that has some similar behaviors as a Python list: you can call `len()` on it, and you can takes slices from it.

Each element in `dataset` is a `Data` object containing a graph that represents a protein. This is the same type of object that we used in the previous part to store the Cora citation network.

Knowing this, answer the following:

- How many proteins (graphs) are there in `dataset`?
- Take any protein from `dataset`. How many nodes and edges does it contain? What is its label? How many features does each node have?

In [ ]:
n_proteins = len(dataset)
prot_i = 18

data = dataset[prot_i]
nnodes = data.num_nodes
in_features = data.num_node_features
is_not_enzyme = data.y.item()

# Just chekin
assert all(d.num_node_features == in_features for d in dataset)

nedges = data.num_edges

# Also possible
assert (nnodes, in_features) == data.x.size()

print(f"""
Number of proteins: {n_proteins}\n
## Protein with index {prot_i}:
\tNumber of nodes: {nnodes}
\tNumber of edges: {nedges}
\tNumber of features per node: {in_features}
\tThe label of protein {prot_i} is: {is_not_enzyme}, so it {'IS NOT' if is_not_enzyme else 'IS'} an enzyme
""")

#### Question 12 (0.5 pt)

To properly train and evaluate our model, we need training, validation, and test splits.

For reproducibility purposes, we generate a random tensor of indices for you. Use it to extract the three splits from `dataset`.

For training, take 80% of the indices (starting from the first element in `indices`), then the following 10% for validation, and the remaining 10% for testing. You can use the indices to index `dataset`.

Call the resulting splits `train_dataset`, `valid_dataset`, and `test_dataset`.

In [ ]:
# Don't erase the following three lines
import torch
import math
torch.random.manual_seed(0)
indices = torch.randperm(len(dataset))

length = len(indices)
print(f"Lenght of the dataset: {length}")
#train_dataset = [dataset[i] for i in indices[:math.floor(length * 0.8)]]

train_dataset = dataset[indices[:math.floor(length * 0.8)]]
#valid_dataset =  [dataset[i] for i in indices[math.floor(length * 0.8):math.floor(length * 0.9)]]

valid_dataset = dataset[indices[math.floor(length * 0.8):math.floor(length * 0.9)]]
#test_dataset = [dataset[i] for i in indices[math.floor(length * 0.9):]]

test_dataset = dataset[indices[math.floor(length * 0.9):]]
print("Lenght of training set: {}, lenght of validation set: {}, lenght of test set: {}".
format(len(train_dataset),len(valid_dataset),len(test_dataset)))

### Working with a batch of graphs

When working with the Cora dataset, you used the information in `data.edge_index` to build the sparse normalized adjacency matrix $\hat{A}$ that is required by the GCN. We could do something similar here: for each graph, we build $\hat{A}$, and pass it to the GCN. However, if the number of graphs is big, this can really slow down training.

To avoid this, we will resort to a very useful trick that also allows us to reuse the same GCN you implemented previously. The trick makes it possible to do a forward pass through the GCN for multiple, disconnected graphs at the same time (instead of only one), much like when you train with mini-batches for other kinds of data.

Let's first revisit the propagation rule of the GCN, $Z = \hat{A}XW$, with an illustration (we have omitted the cells of $X$ and $W$ for clarity):

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/02-gcn-forward.png">

If we have multiple graphs, we can still use the same propagation rule, if we

- Set $\hat{A}$ to be a block diagonal matrix, where the blocks are the different adjacency matrices of the graphs
- Concatenate the feature matrices along the first dimension

This is illustrated in the following figure, for a batch of 3 graphs. Note that the elements outside of the blocks are zero.

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/02-gcn-batch-forward.png">

The resulting adjacency matrix $\hat{A}_B$ can also be built as a sparse matrix, and once we have it together with the concatenated matrix of features, the computation of the graph convolution is exactly the same as before. Note how this trick also allows us to process graphs with different sizes and structures.


#### Question 13 (0.5 pt)


Just as the citation network, the graphs in each of the datasets you created in Question 12 also have an `edge_index` attribute, which can be used to compute the normalized adjacency matrix $\hat{A}$, for each graph.

Reusing your code for Questions 3 and 5, define a function `get_a_norm()` that takes as input an element of a dataset (e.g. `train_dataset[0]`), and returns a `scipy.sparse` matrix containing $\hat{A}$.

Note that an element of a dataset has properties like `num_edges`, `num_nodes`, etc. which you can use here.

In [21]:
# Your answer here
def get_a_norm(data):

    row = data.edge_index[0, :]
    col = data.edge_index[1, :]
    nedges = data.num_edges
    nnodes = data.num_nodes
    assert len(row) == len(col) == nedges

    # We just put ones for neighbours
    values = torch.ones(nedges)

    shape = (nnodes, nnodes)

    adj_mat = scipy.sparse.coo_matrix((values, (row, col)), shape=shape)

    # Add self-loops. This will turn A into \tilde{A}
    adj_mat.setdiag(1)

    # Sum along columns to get the diagonal
    diag = adj_mat.sum(axis=1).A1

    # Compute degree matrix
    d = scipy.sparse.diags_array(diag, shape=(nnodes, nnodes))

    # Compute \hat{A}
    return ((d ** -(1/2)) @ adj_mat @ (d ** -(1/2))).tocoo()



#### Question 14 (1 pt)

To prepare the batch of graphs, we need to collect multiple adjacency matrices, feature matrices, and labels.

When using the trick described in the last figure, we see that we have to keep track of when a graph starts and when it ends, so that we can later differentiate the outputs due to $X^{(0)}$, $X^{(1)}$, etc. To achieve this, we will additionally collect a 1D array of batch indices, one for each $X^{(i)}$.

The 1D array has as many elements as rows in $X^{(i)}$, and it is filled with the value $i$ (the position of $X^{(i)}$ in the batch):

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/03-batch-indices.png">

We will later concatenate all the 1D arrays along the first dimension, just as we will do with all the $X^{(i)}$.

Define a function `prepare_graphs_batch()` that takes as input a dataset (e.g. `train_dataset`), and does the following

- Create four empty lists:
  - `adj_matrices`
  - `feature_matrices`
  - `batch_indices`
  - `labels`
- Iterate over the input dataset, getting one graph at a time. At each step, use your function from Question 13 to append the adjacency matrix to `adj_matrices`, append the matrix of input features to `feature_matrices`, create the array of batch indices (as explained above) and append it to `batch_indices`, and append the label of the graph to `labels`. **Make sure to convert the label to float**.
- Once the loop is over, use `scipy.sparse.block_diag()` to build the block diagonal matrix $\hat{A}_B$. Convert it to the COO format, and then use your answer to Question 6 to turn it into a sparse PyTorch tensor.
- Use `torch.cat()` to concatenate the tensors in `feature_matrices` along the first dimension. Do this also for `batch_indices` and `labels`.
- Return the 4 tensors computed in the previous two items.

In [24]:
def prepare_graphs_batch(graphs):
    adj_matrices, feature_matrices, batch_indices, labels = [], [], [], []

    batch_num = 0
    for graph in graphs:
        labels.append(graph.y.float())
        batch_indices.append(torch.tensor([batch_num for _ in range(0,graph.x.shape[0])],dtype=torch.int64))
        feature_matrices.append(graph.x)

        #do the adj_matrix 
        matrix = get_a_norm(graph)
        adj_matrices.append(matrix)
        batch_num += 1
        
    labels = torch.cat(labels,dim=0)
    feature_matrices = torch.cat(feature_matrices,dim=0)
    batch_indices = torch.cat(batch_indices,dim=0)

    # creat block diagonal adjacency matrix
    block_adj = scipy.sparse.block_diag(adj_matrices)

    coords = np.array(block_adj.coords)
    values = block_adj.data
    nnodes = block_adj.shape[0]
    block_adj_torch = torch.sparse_coo_tensor(indices=coords, values=values, size=(nnodes, nnodes), dtype=torch.float)


    return block_adj_torch, feature_matrices, batch_indices, labels



Once your answer for the previous question is ready, you can run the next cell to prepare all the required information, for the train, validation, and test splits.

In [25]:
train_a_norm, train_features, train_batch_idx, train_labels = prepare_graphs_batch(train_dataset)
valid_a_norm, valid_features, valid_batch_idx, valid_labels = prepare_graphs_batch(valid_dataset)
test_a_norm, test_features, test_batch_idx, test_labels = prepare_graphs_batch(test_dataset)

### GCNs for graph classification

We now have all the ingredients to pass a batch of graphs to a GCN. However, for each graph in the batch, the output $Z^{(i)}$ contains one row for each node in the graph. If the goal is to do classification at the graph level, we have to *pool* these vectors to then compute the required logits for classification.

This operation is similar as how pooling works in a CNN. We could consider taking the mean of the vectors, the sum, or use max-pooling. The difference with respect to CNNs is that in our case, we have a batch of graphs, each potentially with a different number of nodes.

To implement this specific pooling, we can use the scatter operation in the `torch_scatter` library, which comes when installing PyG. We will use it, together with the tensor of batch indices from the previous two questions, to pool the outputs of the GCN for each graph, into a single vector:

<img src="https://raw.githubusercontent.com/dfdazac/dlvu-a5/main/img/04-scatter.png">

You can check more details in the [documentation](https://pytorch-scatter.readthedocs.io/en/latest/functions/scatter.html).

#### Question 15 (1.0 pt)

Implement a `GraphClassifier` module using PyTorch.

- The constructor should take as arguments the number of input features, the hidden dimension, and the number of classes.
- The model should contain a instance of the `GCN` module (as you implemented it in Question 8). Use the same value for the hidden dimension and the number of output features (recall that your `GCN` module from Question 8 has two GCN layers).
- The model should also contain a `torch.nn.Linear` layer, with the hidden dimension as the input features, and the number of classes as the output.
- The forward method receives the concatenated matrix of features, the sparse block diagonal adjacency matrix, and the batch indices (the latter is used when calling `scatter`).
- Use the following architecture in the forward pass:
  - GCN $\to$ ReLU $\to$ scatter (max) $\to$ Linear.

The output of the forward should be a 1D tensor (you might need to call `squeeze` to get rid of extra dimensions) containing the logits for all graphs in the batch, for the binary classification task.

In [ ]:
from torch_scatter import scatter


import torch.nn.functional as F

class GraphClassifer(torch.nn.Module):

    def __init__(self, in_features, n_hidden, n_output):
        super().__init__()

        self.gcn = GCN(in_features=in_features, n_hidden=n_hidden, n_output=n_hidden)

        self.fc = torch.nn.Linear(n_hidden, n_output)

    def forward(self, x, adjacency, batch_idx):
        x = self.gcn(x, adjacency)
        x = F.relu(x)
        x = scatter(x, batch_idx, dim=0, reduce='max')
        x = self.fc(x)
        return x.squeeze()
        # return F.log_softmax(x, dim=1).squeeze()



in_features = train_features.shape[1]
graph_classifier = GraphClassifer(in_features=in_features, n_hidden=32, n_output=1)
output = graph_classifier(train_features, train_a_norm, train_batch_idx)
print(output.size())

assert output.size()[0] == len(train_dataset)


#### Question 16 (1.5 pt)

Implement a training loop for the graph classifier. Use the data from Question 14 to train and evaluate the model.

We encourage you to use a GPU in this section for faster training. Note that if you change the runtime at this point, you must re-execute several of the cells above, including the ones that install PyG.

- Instantiate a classifier with 32 as the hidden dimension
- Use Adam with a learning rate of 1e-3.
- Use `torch.nn.BCEWithLogitsLoss` as the loss function.
- Train for 5,000 epochs. Once training is done, plot the loss curve and the accuracy in the validation set. Then report the accuracy in the test set.

**Note:** the logits from the output of the classifier come from a linear layer. To compute actual predictions for the calculation of the accuracy, pass the logits through `torch.sigmoid()`, and set the predicted values to 1 whenever they are greater than 0.5, and to 0 otherwise.

You should get an accuracy equal to or higher than 70% in the validation and test sets. Can you beat the [state-of-the-art](https://paperswithcode.com/sota/graph-classification-on-proteins)? Feel free to modify your architecture and experiment with it.

Discuss what you observe during training and your results.

In [50]:
# If your runtime is GPU-enabled, use .to(device) to move the model
# and all the relevant tensors to the GPU. You have to move tensors back to CPU
# when computing metrics like accuracy, using .cpu().

# Your answer here

from torch import optim
from torch import nn

def train_graph_classifier(train_a_norm, train_features, train_batch_idx, train_labels,
                           valid_a_norm, valid_features, valid_batch_idx, valid_labels,
                           test_a_norm, test_features, test_batch_idx, test_labels,
                           epochs=5000, learning_rate=1e-3, hidden_dim=32):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Model, Optimizer, Loss
    in_features = train_features.shape[1]
    model = GraphClassifer(in_features=in_features, n_hidden=hidden_dim, n_output=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    # Move data to device
    train_a_norm = train_a_norm.to(device)
    train_features = train_features.to(device)
    train_batch_idx = train_batch_idx.to(device)
    train_labels = train_labels.to(device)
    
    valid_a_norm = valid_a_norm.to(device)
    valid_features = valid_features.to(device)
    valid_batch_idx = valid_batch_idx.to(device)
    valid_labels = valid_labels.to(device)

    test_a_norm = test_a_norm.to(device)
    test_features = test_features.to(device)
    test_batch_idx = test_batch_idx.to(device)
    test_labels = test_labels.to(device)

    # Store data
    loss_train = torch.zeros(epochs)
    loss_val = torch.zeros_like(loss_train)
    acc_train = torch.zeros_like(loss_train)
    acc_val = torch.zeros_like(loss_train)


    # Training loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        output = model(train_features, train_a_norm, train_batch_idx)

        # Loss Computation
        loss = criterion(output, train_labels)

        # Backprop
        loss.backward()
        optimizer.step()

        # Metrics
        loss_train[epoch] = loss.item()
        predicted_train = torch.sigmoid(output)
        acc_train[epoch] = ((predicted_train>0.5)==train_labels).float().mean()

        # Validation
        model.eval()
        with torch.no_grad():
            output_val = model(valid_features, valid_a_norm, valid_batch_idx)
            loss_val[epoch] = criterion(output_val, valid_labels).item()
            predicted_val = torch.sigmoid(output_val)
            acc_val[epoch] = ((predicted_val>0.5)==valid_labels).float().mean()
        print(f"Epoch {epoch+1}\n-------------------------------")
        print(f"Training loss: {loss_train[epoch]:>5f} \n Training accuracy: {acc_train[epoch]:>5f}\n Validation accuracy: {acc_val[epoch]:.4f}")

    print("Finished Training")

    # Test
    model.eval()
    with torch.no_grad():
        test_output = model(test_features, test_a_norm, test_batch_idx)
        test_predicted = torch.sigmoid(test_output)
        test_acc = ((test_predicted>0.5)==test_labels).float().mean()
        print(f"Test Accuracy: {test_acc:.4f}")
    
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    ax[0].plot(range(epochs), loss_train, label="Training loss")
    ax[0].plot(range(epochs), loss_val, label="Validation loss")
    ax[0].set_title("Loss per epoch")
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("Loss")
    ax[0].legend()

    ax[1].plot(range(epochs), acc_train, label="Acc. train")
    ax[1].plot(range(epochs), acc_val, label="Acc. val")
    ax[1].set_title("Accuracy per epoch")
    ax[1].set_xlabel("Epochs")
    ax[1].set_ylabel("Accuracy")
    ax[1].legend()

    plt.show()


train_graph_classifier(train_a_norm, train_features, train_batch_idx, train_labels,
                           valid_a_norm, valid_features, valid_batch_idx, valid_labels,
                           test_a_norm, test_features, test_batch_idx, test_labels,
                           epochs=5000, learning_rate=1e-3, hidden_dim=32)


## The end

If you have made it all the way here successfully, congratulations! 🎉 

You have implemented your own GCN and tested it on a node classification task, and a more challenging classification task over multiple graphs.

We hope you can use this knowledge to apply GCNs not only to the tasks described here, but other applications where data can be modeled as a graph.

If you are interested in applying graph neural networks to larger graphs, or try newer architectures, you can dive deeper into [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/), a library with fast implementations for a wide range of architectures. It also comes with custom code that takes care of aspects that you dealt with manually for this assignment, like a more efficient implementation of the adjacency matrix multiplication via message-passing methods, and Data Loaders that relieve you from having to build block diagonal sparse matrices.

You can also check the [Deep Graph Library](https://docs.dgl.ai/) another powerful library for deep learning on graphs which also integrates with other backends like TensorFlow.

# Grading (10pt)

- Question 1: 0.25pt 
- Question 2: 0.25pt 
- Question 3: 0.5pt 
- Question 4: 0.25pt 
- Question 5: 0.5pt 
- Question 6: 0.5pt 
- Question 7: 0.5pt 
- Question 8: 0.5pt 
- Question 9: 1.5pt 
- Question 10: 0.5pt 
- Question 11: 0.25pt 
- Question 12: 0.5pt 
- Question 13: 0.5pt 
- Question 14: 1pt
- Question 15: 1pt
- Question 16: 1.5pt